# **Find the sensor ID**

In [ ]:
import requests
# Mosquito Fire : -120.7447,39.00591
url = "https://api.purpleair.com/v1/sensors"

querystring = { "fields": "latitude,altitude,longitude", "location_type": "0", "nwlng": "-120.84", "nwlat": "39.1", "selng": "-120.64", "selat": "38.9" }

headers = { "X-API-Key": "YOUR API KEY" }

response = requests.get(url, headers=headers, params=querystring)

print(response.text)

# **Convert Unix Timestamp to a regular datetime**

In [ ]:
import datetime

start_date = '2022-09-06'
end_date ='2022-09-19'
date_format = '%Y-%m-%d'
start_date_unix = datetime.datetime.strptime(start_date, date_format)
end_date_unix = datetime.datetime.strptime(end_date, date_format)
print('start_date=',int(start_date_unix.timestamp()),'end_date=',int(end_date_unix.timestamp()))

# **Downlaod the PurpleAir data**

In [ ]:
import requests
from io import StringIO
import pandas as pd

sensor_ID = '8092'  # 111415, 116649, 158587, 79615, 8092
url = f"https://api.purpleair.com/v1/sensors/{sensor_ID}/history/csv"

headers = {
    "X-API-Key": "YOUR API KEY"
}
params = {
    "start_timestamp": "1662447600",
    "end_timestamp": "1663570800",
    "average": "60",
    "fields": "pm2.5_atm"
}

response = requests.get(url, headers=headers, params=params)

# Access the response content
#print(response.content)
df = pd.read_csv(StringIO(response.content.decode()), sep=',')

# Convert Unix timestamps to date strings with hours
df['date_string'] = df['time_stamp'].apply(lambda x: datetime.datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
df_sorted = df.sort_values(by='date_string', key=lambda x: pd.to_datetime(x))

# Convert the 'date_string' column to datetime format
df_sorted['date_string'] = pd.to_datetime(df_sorted['date_string'])

# Extract the 'yyyy-mm-dd' part from the 'date_string' column
df_sorted['date'] = df_sorted['date_string'].dt.date
# Save the dataframe in a CSV
df_sorted.to_csv(f'{sensor_ID}.csv', index=False)

# **Plot the PurpleAir data**

In [ ]:
from numpy.core.multiarray import datetime_as_string
import matplotlib.pyplot as plt

# Read the CSV files (PurpleAir)
df_111415 = pd.read_csv('111415.csv')
df_116649= pd.read_csv('116649.csv')
df_158587 = pd.read_csv('158587.csv')
df_79615= pd.read_csv('79615.csv')
df_8092= pd.read_csv('8092.csv')

# Filter the Dataframe
cols=['sensor_index','date_string', 'pm2.5_atm']
df_pureple_air = pd.concat([df_111415[cols], df_116649[cols], df_158587[cols], df_79615[cols], df_8092[cols]], ignore_index=True)
df_pureple_air['date_string'] = pd.to_datetime(df_pureple_air['date_string'])

# Plotting the data
plt.figure(figsize=(10, 6))
for sensor_id, group in df_pureple_air.groupby('sensor_index'):
    plt.plot(group['date_string'], group['pm2.5_atm'], marker='o', linestyle='-', label=sensor_id)

# Adding labels and title
plt.xlabel('Date')
plt.ylabel('PM 2.5')
plt.title('Timeseries of PurepleAir Sensors')
plt.legend()
plt.show()